# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Краткое описание 

Наша задача - ответиь на следующие вопросы:
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

Для выполнения этой задачи нам предоставлен датасет. План работы стандартный: 
- открыть,
- обработать,
- категоризировать,
- ответить на вопросы, 
- сделать выводы.

# Оглавление
1. [Шаг 1. Откройте файл с данными и изучите общую информацию](#start)
2. [Шаг 2. Предобработка данных](#preprocessing)
3. [Шаг 3. Ответьте на вопросы](#question)
4. [Шаг 4. Общий вывод](#conclusion)
5. [Чек-лист готовности проекта](#list)

## Шаг 1. Откройте файл с данными и изучите общую информацию 
<a id="start"></a>

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem() 

In [2]:
df = pd.read_csv('/datasets/data.csv')
proba = df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB



<h3> Вывод по info() </h3>

В таблице 12 столбцов. Типы данных в столбцах: object, int64,float64.

В названиях колонок нарушениq нет, все записаны в змеином регистре. Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.
</div>

In [3]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
df.tail(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [5]:
df.sample(n = 10, random_state = 42)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
423,0,-191.167760,56,среднее,1,женат / замужем,0,M,сотрудник,0,138653.748793,автомобили
3522,0,-2319.817259,58,среднее,1,Не женат / не замужем,4,F,сотрудник,0,123152.627177,покупка жилья
8760,1,-2990.578297,34,высшее,0,гражданский брак,1,M,компаньон,0,232380.737167,свадьба
20695,0,-926.209452,28,среднее,1,Не женат / не замужем,4,F,сотрудник,0,210617.086601,получение высшего образования
4351,0,-2524.302106,42,СРЕДНЕЕ,1,женат / замужем,0,F,сотрудник,0,78487.540219,сделка с автомобилем
3002,0,NaN,28,высшее,0,женат / замужем,0,M,компаньон,0,NaN,покупка жилой недвижимости
17306,0,-5000.830630,0,среднее,1,гражданский брак,1,F,компаньон,0,83019.106740,свадьба
2830,0,-574.995923,40,среднее,1,женат / замужем,0,F,сотрудник,0,161158.806308,автомобили
14822,2,NaN,33,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,заняться образованием
10347,1,-713.140231,48,среднее,1,женат / замужем,0,M,компаньон,0,177474.885995,ремонт жилью


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06



<h3> Вывод по изначальным данным</h3>

В каждой строке таблицы — данные о клиенте банка. Часть колонок описывает самого клиента: пол, возраст, кол-во детей, образование.  Остальные данные расскрывают финансовую характеристику клиента: имел ли задолженность по возврату кредитов, ежемесячный доход,
цель получения кредита

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, странные значения в колонке общего трудового стажа (отрицательные значения, огромные значения), написанные капсом значения в колонке education.

Чтобы двигаться дальше, нужно устранить проблемы в данных.


<a id="preprocessing"></a>
## Шаг 2. Предобработка данных 

### Обработка пропусков

Пропуска в столбцах 'days_employed' , и 'total_income'

In [7]:
df['days_employed'].isna().sum()

2174

In [8]:
df['total_income'].isna().sum()

2174

In [9]:
df[df['total_income'].isna()].head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Кол-во пропусков одинаковое. Удалять строки с пустыми значениями нельзя так 2174 от 21525 это примерно 10%. Удалив строки, мы потеряем важную часть данных. В столбце days_employed пропущенные значения заменим на кол-во дней, если бы  клиент начал работать с 18 лет и до сегоднешнего дня. В столбце total_income	заменим пропуски на медианный доход. Приступим+)

In [10]:
# Функция для заполнения пустых значений в доходе 
# исходя от возраста
def vos_doxod(row):
    doxod = row['total_income']
    years = row['dob_years']
    if pd.isna(doxod) == True:
        if 18<years<25:
            doxod = 80000
        elif 25<=years<40:
            doxod = 150000
        elif 40<=years<100:
            doxod = 90000
    return doxod



In [11]:
df['total_income'] = df.apply(vos_doxod, axis = 1) # применяем функцию

In [12]:
df.sample(n = 10, random_state = 42) # проверяем

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
423,0,-191.167760,56,среднее,1,женат / замужем,0,M,сотрудник,0,138653.748793,автомобили
3522,0,-2319.817259,58,среднее,1,Не женат / не замужем,4,F,сотрудник,0,123152.627177,покупка жилья
8760,1,-2990.578297,34,высшее,0,гражданский брак,1,M,компаньон,0,232380.737167,свадьба
20695,0,-926.209452,28,среднее,1,Не женат / не замужем,4,F,сотрудник,0,210617.086601,получение высшего образования
4351,0,-2524.302106,42,СРЕДНЕЕ,1,женат / замужем,0,F,сотрудник,0,78487.540219,сделка с автомобилем
3002,0,NaN,28,высшее,0,женат / замужем,0,M,компаньон,0,150000.000000,покупка жилой недвижимости
17306,0,-5000.830630,0,среднее,1,гражданский брак,1,F,компаньон,0,83019.106740,свадьба
2830,0,-574.995923,40,среднее,1,женат / замужем,0,F,сотрудник,0,161158.806308,автомобили
14822,2,NaN,33,среднее,1,женат / замужем,0,F,сотрудник,0,150000.000000,заняться образованием
10347,1,-713.140231,48,среднее,1,женат / замужем,0,M,компаньон,0,177474.885995,ремонт жилью


In [13]:
df['days_employed'] = df['days_employed'].fillna((df['dob_years'] - 18) * 365)


In [14]:
med_income =  df['total_income'].median() # находим медианный доход
df['total_income'] = df['total_income'].fillna(med_income) # Заменяем пропущенные значения на медианный доход
df.head(14) # Проверяем

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В столбце days_employed	 - странные значения. Причем одна часть вроде бы подходит (просто отрицательаня)(, в другой чатсь непомерно большие значения. Отрицательные значения заменим на положительные (abs). С большими значениями применем метод, как делали и с пропусками. В идеале подойти к сотруднику отвечающий за изначальные данные и спросить по какой формуле считался трудовой стаж, но такой возможности нет=( Так что работаеи с тем, что есть =)

Проверка подходят ли нам отрицательные значения (50 лет стажа = 18250 дней).

In [15]:
df[df['days_employed'] < -18250] #  Подходят

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью


In [16]:
df['days_employed'] = df['days_employed'].abs() # Делаем все положительным
df.head(14) #Проверяем

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Проверяем не попадаются ли промежуточные большие значения.

In [17]:
df[(df['days_employed'] < 300000) & (df['days_employed'] > 20000)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
18664,1,20075.0,73,среднее,1,женат / замужем,0,F,пенсионер,0,90000.0,дополнительное образование


Все хорошо. Значит работаем со значениями, которые только больше 300000. Напишем функцию, которая будет проверять: если значение  в days_employed больше 300000, то перводим в максимальный возможный трудовой стаж, если бы человек продолжает работать и начал с 18 лет.

In [18]:
def right_days(row):
    days = row['days_employed']
    years = row['dob_years']
    if days > 300000:
        days = (years - 18) * 365
    elif days == 0: # В ходе проверок было обнаружен нулевое значение
        days = 18250
    return days

In [19]:
df['days_employed'] = df.apply(right_days, axis = 1)
df[(df['days_employed'] < 300000) & (df['days_employed'] > 21000)]# Проверяем

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose



**Вывод по обработке пропусков**
    
В данном блоке я разобрал не только пропуска ну и некоректные значения в трудовом стаже. Пропусков больше нет. Получен приближенный к жизни трудовой стаж. Важно узнать как и кто рассчитывал эти значения, чтоб в будущем получать корректныне данные.


### Замена типа данных

Заменим в столбцах 'days_employed' и total_income тип данных с float64 на int. Так как нам в принципе не нужны дробные числв в этих столбцах. Используем метод astype('int')

In [22]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [23]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()

In [24]:
df['dob_years'] = df['dob_years'].astype('uint8') # нам хватит и  uint8
df['days_employed'] = df['days_employed'].astype('uint') # Здесь невозможны отрицательные числа
df['total_income'] = df['total_income'].astype('uint' )# Здесь невозможны отрицательные числа
df['family_status'] = df['family_status'].astype('category') # Превращаем в категорию
df['education'] = df['education'].astype('category')  # Превращаем в категорию
df['income_type'] = df['income_type'].astype('category')

In [25]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [26]:
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 1)

In [27]:
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [28]:
df['children'] = df['children'].astype('uint8') # нам хватит и uint8

**Вывод**

В столбцах были заменены типы данных, а именно были использованы типы данных category, uint . Это позволило сэконмить 41 процент памяти, что и  уменьшит время обработки данных компьютером.

### Обработка дубликатов

**Старый Вывод**

В столбцах 'days_employed' и total_income тип данных заменен с float64 на int. Это уменьшит время обработки данных компьютером.

Приведем все к нижнему регистру методом str.lower(). и пройдемся по всем нужнам нам столбцам.

In [31]:
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

В столбце обнаружены неправильные значения, а именно -1 и 20. Устраним их:
- Отрицательное превартим в положительное
- 20 превратим в 20

In [32]:
def deti(kol_vo):
    if kol_vo == 20:
        kol_vo = 2
    return kol_vo

In [33]:
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [34]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [35]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [36]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [37]:
df.duplicated().sum()

71

In [38]:
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

0

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null uint8
days_employed       21454 non-null uint64
dob_years           21454 non-null uint8
education           21454 non-null category
education_id        21454 non-null int64
family_status       21454 non-null category
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null category
debt                21454 non-null int64
total_income        21454 non-null uint64
purpose             21454 non-null object
dtypes: category(3), int64(3), object(2), uint64(2), uint8(2)
memory usage: 1.2+ MB


**Вывод**

Мы привели все к нижнему регистру, поправили столбец с детьми, избавились от дубликатов. Дубликаты могли появится из-за неправильной выгрузки данных или при объединении таблиц с данными. Кол-во строк в новой таблице 21454.

### Лемматизация

Для групировки целей для кредита нам необходимо провести Лемматизацию

In [40]:
def lema(text):
    l_text = m.lemmatize(text)
    return l_text

In [41]:
df['l_purpose'] = df['purpose'].apply(lema)
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,l_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,12775,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]"
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]"
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]"
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"


In [42]:
## Другой способ лемматизации через уникальные значения и merge
unique_purpose =proba ['purpose'].unique()
lemass = []
for kek in unique_purpose:
    lemass.append(m.lemmatize(kek))

In [43]:
#new = pd.concat([lemass, unique_purpose], axis=1)
unique_purpose1 = pd.Series(unique_purpose)
lemass1 = pd.Series(lemass)
new = pd.concat([unique_purpose1,lemass1], axis=1)
new.columns = ['purpose', 'l_purpose']
new


,purpose,l_purpose
0,покупка жилья,"[покупка, , жилье, \n]"
1,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,дополнительное образование,"[дополнительный, , образование, \n]"
3,сыграть свадьбу,"[сыграть, , свадьба, \n]"
4,операции с жильем,"[операция, , с, , жилье, \n]"
5,образование,"[образование, \n]"
6,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
7,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"
8,покупка недвижимости,"[покупка, , недвижимость, \n]"
9,покупка коммерческой недвижимости,"[покупка, , коммерческий, , недвижимость, \n]"


In [44]:
proba.merge(new, how = 'left')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,l_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,12775,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]"
21521,0,17885,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]"
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]"
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]"


**Вывод**

Мы создали новый столбец, где у нас хранятся лематизированные слова из столбца с целью, и теперь нам будет удобно разбить цели на разные категории

### Категоризация данных

Сделаем категории и добавим столбцы для облегчения категоризации данных. Итак нам пондобиться категории:
- наличие детей
- уровень дохода
- цели кредита
Для этого мы создадим три функции

In [45]:
# Функция для наличия детей
def nalich_deti(kol_vo):
    if kol_vo != 0:
        return "есть ребенок/дети"
    elif kol_vo == 0: 
        return "нет детей"

In [46]:
df['deti'] = df['children'].apply(nalich_deti)
df['deti'].value_counts()

нет детей            14091
есть ребенок/дети     7363
Name: deti, dtype: int64

In [47]:
df['total_income'].max()

2265604

In [48]:
df['total_income'].min()

20667

In [49]:
df['total_income'].median()

142530.0

 Разобьем на три группы уровня дохода: низкий, средний, вывсокий
За средний уровень возьмем от 100000 до 200000. Средний доход по официальным данным за 2021 =  58782 рубля, что немного грустно, и нам не подойдет.

In [50]:
# Функция для уровня дохода
def doxod(dengi):
    if  0 <= dengi <= 100000:
        return "низкий доход"
    elif 100000 <= dengi <= 200000:
        return "средний доход"
    else:
        return "высокий доход"

In [51]:
df['level_doxod'] = df['total_income'].apply(doxod)
df['level_doxod'].value_counts()

средний доход    10585
низкий доход      5803
высокий доход     5066
Name: level_doxod, dtype: int64

Согласно результатам выше, мы можем выделить следующие цели для кредита:
- Для недвижимости
- Для автомобиля
- Для образования
- Для свадьбы

In [52]:
# Функция на цель
def goal(spisok):
    for text in spisok:
        if text == 'жилье' or text == 'недвижимость':
            return "на недвижимость"
        elif text == 'автомобиль' or text == 'машина':
            return "на автомобиль"
        elif text == 'образование':
            return "на образование"
        elif text == 'свадьба':
            return "на свадьбу"

In [53]:
df['goal'] = df['l_purpose'].apply(goal)
df['goal'].value_counts()

на недвижимость    10811
на автомобиль       4306
на образование      4013
на свадьбу          2324
Name: goal, dtype: int64

**Вывод**

Категории сделаны можно приступать к анализу зависимости.

<a id="question"></a>
## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [54]:
vse_dolgniki = len(df) # общее кол-во людей в базе
vse_dolgniki

21454

In [55]:
pd.options.display.float_format = '{:.2f}%'.format # Это чтобы знаков после запятой немного было в таблицах

In [56]:
df['debt'] = df['debt'].replace(0, 'нет задолжностей') # Для красоты таблиц следующих
df['debt'] = df['debt'].replace(1, 'была задолжность')

In [63]:
df_deti = df.groupby(['deti','debt'])['debt'].count()
df_deti1 = df_deti / vse_dolgniki * 100
df_deti3 = df[['deti','debt']].agg(['count'])
df_deti

deti               debt            
есть ребенок/дети  была задолжность      678
                   нет задолжностей     6685
нет детей          была задолжность     1063
                   нет задолжностей    13028
Name: debt, dtype: int64

In [59]:
df_deti2 = pd.concat([df_deti,df_deti1], axis =1)
#df_deti2.append(df_deti32)
#df_deti4 = pd.concat([df_deti,  df_deti32], axis =1)
df_deti2.columns= ['кол-во клиентов',  '% от общего кол-ва клиентов']
df_deti2

кол-во клиентов  \
deti              debt                                
есть ребенок/дети была задолжность              678   
                  нет задолжностей             6685   
нет детей         была задолжность             1063   
                  нет задолжностей            13028   

                                    % от общего кол-ва клиентов  
deti              debt                                           
есть ребенок/дети была задолжность                        3.16%  
                  нет задолжностей                       31.16%  
нет детей         была задолжность                        4.95%  
                  нет задолжностей                       60.73%

**Вывод**

Наличие детей сказывается на выплату кредита. Есть задлжности больше у кого есть ребенок

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [60]:
df_family = df.groupby(['family_status','debt'])['debt'].count()
df_family1 = df_family / vse_dolgniki * 100
df_family2 = pd.concat([df_family, df_family1], axis =1)
df_family2.columns= ['кол-во клиентов',  '% от общего кол-ва клиентов']
df_family

family_status          debt            
в разводе              была задолжность       85
                       нет задолжностей     1110
вдовец / вдова         была задолжность       63
                       нет задолжностей      896
гражданский брак       была задолжность      388
                       нет задолжностей     3763
женат / замужем        была задолжность      931
                       нет задолжностей    11408
не женат / не замужем  была задолжность      274
                       нет задолжностей     2536
Name: debt, dtype: int64

**Вывод**

Неженатые и незамужние люди чаще других задерживают платежку

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [61]:
df_dengi = df.groupby(['level_doxod','debt'])['debt'].count()
df_dengi1 = df_dengi / vse_dolgniki * 100
df_dengi2 = pd.concat([df_dengi, df_dengi1], axis =1)
df_dengi2.columns= ['кол-во клиентов',  '% от общего кол-ва клиентов']
df_dengi2

кол-во клиентов  % от общего кол-ва клиентов
level_doxod   debt                                                          
высокий доход была задолжность              358                        1.67%
              нет задолжностей             4708                       21.94%
низкий доход  была задолжность              446                        2.08%
              нет задолжностей             5357                       24.97%
средний доход была задолжность              937                        4.37%
              нет задолжностей             9648                       44.97%

**Вывод**

Клиенты со высоким и низким доходом реже имеют задолжности.

- Как разные цели кредита влияют на его возврат в срок?

In [62]:
df_goal = df.groupby(['goal','debt'])['debt'].count()
df_goal1 = df_goal / vse_dolgniki * 100
df_goal2 = pd.concat([df_goal, df_goal1], axis =1)
df_goal2.columns= ['кол-во клиентов',  '% от общего кол-ва клиентов']
df_goal2

кол-во клиентов  % от общего кол-ва клиентов
goal            debt                                                          
на автомобиль   была задолжность              403                        1.88%
                нет задолжностей             3903                       18.19%
на недвижимость была задолжность              782                        3.65%
                нет задолжностей            10029                       46.75%
на образование  была задолжность              370                        1.72%
                нет задолжностей             3643                       16.98%
на свадьбу      была задолжность              186                        0.87%
                нет задолжностей             2138                        9.97%

**Вывод**
Клинты, взявшие кредит на недвижимость, реже имеют задолжности чем все остальные.

<a id="conclusion"></a>
##  Общий вывод

Полученные от заказчика данные были обработаны, а именно: обработаны пустые ячейки, удалены дубликаты, категоризированы. Самыми порядочными плательщакми оказываюся свободные люди без детей, имеющие низкий или высокий доход, взявшие кредит на недвижимость.